# Quadratic Assignment Problem Sample

**Workspace Setup**  
An Azure Quantum workspace is needed for this sample. You will need to enter your Azure Quantum workspace details in the cell below before you submit a problem:

In [ ]:
# You may need to upgrade your SDK to a version that supports SlcTerm
!pip install azure-quantum --upgrade

In [7]:
# This allows you to connect to the Workspace you've previously deployed in Azure.
# Be sure to fill in the settings below which can be retrieved by running 'az quantum workspace show' in the terminal.
from azure.quantum import Workspace

# Copy the settings for your workspace below
workspace = Workspace (
  subscription_id = "",
  resource_group = "",
  name = "",
  location = ""
)

In [ ]:
# Import additional packages
import time
from typing import List
from azure.quantum.optimization import PopulationAnnealing
from azure.quantum.optimization import Problem, ProblemType, Term, SlcTerm, GroupType

# Quadratic Assignment Problem

## Introduction
The quadratic assignment problem (QAP) is an NP-hard problem involving the assignment of facilities onto different locations. As quoted from [Wikipedia](https://en.wikipedia.org/wiki/Quadratic_assignment_problem): 

*There are a set of **n facilities** and a set of **n locations**. For each pair of locations, a distance is specified and for each pair of facilities a weight or flow is specified (e.g., the amount of supplies transported between the two facilities). The problem is to assign all facilities to different locations with the goal of minimizing the sum of the distances multiplied by the corresponding flows.*

### Example Problem
Consider the following small example with 3 locations $\{l_{0}, l_{1}, l_{2}\}$ and 3 facilities $\{f_{0} = school,  f_{1} = playground,  f_{2} = hospital \}$. 

Suppose we are town planner deciding where to put these facilities in the town. We can assign one facility to each location on the map. The connection between each location represents the time taken to travel between the locations.   

![title](media/qap_map1.jpg)

In order to make this decision, we also need to know the expected "throughput" (i.e. persons travelling) between each facility. Suppose we are given the expected number of people travelling between facilities:

||School|Playground|Hospital|
|-|-----|----------|--------|
|School|0|100|0|
|Playground|100|0|5|
|Hospital|0|5|0|

In this example, the expected persons travelling between facilities is symmetrical (although it may not be in another problem). 

### Example Assignment Configuration
We want to place the facilities such that the overall expected travel time (cost) is minimized. An example of a configuration (not necessarily optimal) can be:
$\{school: 2, playground: 1, hospital: 0\}$

With this configuration, the total cost becomes 0\*20 + 5\*2 + 100\*14 = 1410

### Optimal Configuration
Since the problem is small, we can see that the optimal assignment is $\{school: 0, playground: 1, hospital: 2\}$.   

This results in a total cost of 100\*2 + 0\*20 + 5\*14 = 270


### Code
We can define the problem above as two adjacency matrix representations.

|p |School|Playground|Hospital||t|Location 0|Location 1|Location 2|
|-|-----|----------|--------|-|-|-----|----------|--------|
|School|0|100|0| |**Location 0**|0|2|20|
|Playground|100|0|5| |**Location 1**|2|0|14|
|Hospital|0|5|0| |**Location 2**|20|14|0|

In [2]:
locations = [
    [0, 2, 20],
    [2, 0, 14],
    [20, 14,0]
]

facilities = [
    [0, 100, 0],
    [100, 0, 5],
    [0, 5, 0]
]

## Converting QAP to QUBO

The quadratic assignment problem is not natively an unconstrained binary problem. In order to convert this to QUBO form, we need to define an encoding strategy.  


### Variable Definition
One way to do this is to define the following binary variable $x_{i,j}$ that represents whether facility $i$ is assigned to location $j$. 

For the above case of 3 facilities and 3 locations, we have the following 9 binary variables:

||School|Playground|Hospital|
|-|-----|----------|--------|
|Location 0|$x_{0,0}$|$x_{1,0}$|$x_{2,0}$|
|Location 1|$x_{0,1}$|$x_{1,1}$|$x_{2,1}$|
|Location 2|$x_{0,2}$|$x_{1,2}$|$x_{2,2}$|

Where $x_{i,j}$ is 1 if assigned, 0 if not.

- This formulation assumes a symmetric flow between different locations and different facilities. 
- The number of binary variables increases quadratically with the original number of locations and facilities. 


### Objective Function and Coefficient Definition
Recall that we have 2 matrices that define the neighbouring flows between locations (t = time to travel) and facilities (p = persons travelling). 

|p |School|Playground|Hospital||t|Location 0|Location 1|Location 2|
|-|-----|----------|--------|-|-|-----|----------|--------|
|School|0|100|0| |**Location 0**|0|2|20|
|Playground|100|0|5| |**Location 1**|2|0|14|
|Hospital|0|5|0| |**Location 2**|20|14|0|


The expression for the total "cost" of the system (in this case total travel time) can be written as:

$$ Minimize \quad Cost = \sum_{i,j,k!=i,l!=j} p_{i,k} \cdot t_{j,l} \cdot x_{i,j}\cdot x_{k,l}$$


**Intuition**  
Consider a decision variable $x_{0,0}$ (representing **school** assigned to **location 0**). We want consider its cost contribution when placed next to all other possible assignments that **do not include school and location 0**. 

The 4 variables that fit this criteria are: $x_{1,1},x_{1,2},x_{2,1},x_{2,2}$

Considering the effect of $x_{0,0}$ and $x_{1,2}$, we multiply the connections between their respective facilities and locations. Here school (facility 0) is placed next to playground (facility 1) which has a 100 weight. The cost between location 0 and 2 is 20. The total weight of this edge is 20\*100 = 2000. 

Repeat this for every  $x_{i,j}$ and we have the objective function. 

### Code
The code snippet below constructs the terms for this example using the two matrices and variable definition above.

In [11]:
# variable ids
# we will assign a numerical id to each x_i,j variable for a total of 9 variables
n = 3
variable_map = {}
for i in range(n):
    for j in range(n):
        variable_map[(i,j)] = i*n + j

terms_list = [] 

# iterate through each variable and build its neighbours
# note: we are not condensing common terms right now.
for x1 in variable_map: 
    i,j = x1
    
    # neighbours are other variables that do not share the same location/facility as the current one
    neighbours = [var for var in variable_map if var[0] != i and var[1] != j]
    for x2 in neighbours:
        i2,j2 = x2
        c = locations[j][j2]*facilities[i][i2]/2.0 # divide by 2 because of symmetrical matrix
        terms_list.append(Term(c=c, indices=[variable_map[x1], variable_map[x2]]))

print("variables -> ids:", variable_map)
print("\nterms:", terms_list)

variables -> ids: {(0, 0): 0, (0, 1): 1, (0, 2): 2, (1, 0): 3, (1, 1): 4, (1, 2): 5, (2, 0): 6, (2, 1): 7, (2, 2): 8}
terms: [{'c': 100.0, 'ids': [0, 4]}, {'c': 1000.0, 'ids': [0, 5]}, {'c': 0.0, 'ids': [0, 7]}, {'c': 0.0, 'ids': [0, 8]}, {'c': 100.0, 'ids': [1, 3]}, {'c': 700.0, 'ids': [1, 5]}, {'c': 0.0, 'ids': [1, 6]}, {'c': 0.0, 'ids': [1, 8]}, {'c': 1000.0, 'ids': [2, 3]}, {'c': 700.0, 'ids': [2, 4]}, {'c': 0.0, 'ids': [2, 6]}, {'c': 0.0, 'ids': [2, 7]}, {'c': 100.0, 'ids': [3, 1]}, {'c': 1000.0, 'ids': [3, 2]}, {'c': 5.0, 'ids': [3, 7]}, {'c': 50.0, 'ids': [3, 8]}, {'c': 100.0, 'ids': [4, 0]}, {'c': 700.0, 'ids': [4, 2]}, {'c': 5.0, 'ids': [4, 6]}, {'c': 35.0, 'ids': [4, 8]}, {'c': 1000.0, 'ids': [5, 0]}, {'c': 700.0, 'ids': [5, 1]}, {'c': 50.0, 'ids': [5, 6]}, {'c': 35.0, 'ids': [5, 7]}, {'c': 0.0, 'ids': [6, 1]}, {'c': 0.0, 'ids': [6, 2]}, {'c': 5.0, 'ids': [6, 4]}, {'c': 50.0, 'ids': [6, 5]}, {'c': 0.0, 'ids': [7, 0]}, {'c': 0.0, 'ids': [7, 2]}, {'c': 5.0, 'ids': [7, 3]}, {'c'

## Constraints and Penalty Terms

You might have noticed that the objective function definition described above is missing a crucial component - that is, the additional constraints that arise from the binary formulation. 

The main constraints in the original problem that are lost in the above binary formulation are the following:
- facilities cannot be reused
- one location must hold exactly one facility (not more, not less)

To translate this into mathematical expressions, we have:

$$  \sum_{i} x_{i,k} = 1 \quad \text{for all k}$$
$$  \sum_{i} x_{k,i} = 1 \quad \text{for all k}$$

In other words, only one of each variable that contains the same location (or facility) can be committed at a time. All other cases are invalid. 

In order to enforce this constraint in the binary objective function above, we introduce the concept of **penalty terms**.

### Penalty terms
Penalty terms are additional terms on top of the objective function that represent the constraints. They increase the objective value if a constraint has been violated. Since the constraints for this problem are all equality constraints, a penalty term is a squared linear combination of the associated variables. 

$$  P \cdot (\sum_{i} x_{i,k} - 1)^2 \quad \text{for all k}$$
$$  P \cdot (\sum_{i} x_{k,i} - 1)^2 \quad \text{for all k}$$

The penalty (P) value is configurable and should be chosen such that the terms have enough impact on the cost function (so we get a feasible solution), but not too large that the barriers result in being stuck at a local minima. 


### Code
The code uses a new feature on Azure Quantum called the SlcTerm (Squared Linear Combination) that allows you to specify the equality penalty in its factored form. You can also opt to expand out the raw monomial terms (although that will use more space and increase the size of the final problem). 

The SlcTerm supports an expression in the form:

$$  P \cdot (\sum_{i} c_{i} x_{i} + k)^2 \quad $$

In our example, $k = -1$ and $c_i = 1$

In [12]:
# here we set penalty to the largest coefficient in the original objective function (although we can go higher than this)
P = 1000
k = -1

# construct facility constraints
for i in range(n):    
    terms_list.append(SlcTerm(
        c = P,
        terms = [
            Term(
                c = 1,
                indices = [variable_map[(i,j)]]
            )
        for j in range(n)
        ] + 
        [Term(c=k, indices=[])] #constant k term
    ))

# construct location constraints
for i in range(n):    
    terms_list.append(SlcTerm(
        c = P,
        terms = [
            Term(
                c = 1,
                indices = [variable_map[(j,i)]]
            )
        for j in range(n)
        ] + 
        [Term(c=k, indices=[])] #constant k term
    ))

# construct final problem with all the terms (monomial and slc)
problem = Problem(name="Small QAP", problem_type=ProblemType.pubo, terms=terms_list)

print(f"Number of monimial terms: {len(problem.terms)}, number of grouped terms: {len(problem.terms_slc)}")
print(f"\nGrouped terms: {problem.terms_slc}")

Number of monimial terms: 36, number of grouped terms: 6
Grouped terms: [{'c': 1000, 'terms': [{'c': 1, 'ids': [0]}, {'c': 1, 'ids': [1]}, {'c': 1, 'ids': [2]}, {'c': -1, 'ids': []}]}, {'c': 1000, 'terms': [{'c': 1, 'ids': [3]}, {'c': 1, 'ids': [4]}, {'c': 1, 'ids': [5]}, {'c': -1, 'ids': []}]}, {'c': 1000, 'terms': [{'c': 1, 'ids': [6]}, {'c': 1, 'ids': [7]}, {'c': 1, 'ids': [8]}, {'c': -1, 'ids': []}]}, {'c': 1000, 'terms': [{'c': 1, 'ids': [0]}, {'c': 1, 'ids': [3]}, {'c': 1, 'ids': [6]}, {'c': -1, 'ids': []}]}, {'c': 1000, 'terms': [{'c': 1, 'ids': [1]}, {'c': 1, 'ids': [4]}, {'c': 1, 'ids': [7]}, {'c': -1, 'ids': []}]}, {'c': 1000, 'terms': [{'c': 1, 'ids': [2]}, {'c': 1, 'ids': [5]}, {'c': 1, 'ids': [8]}, {'c': -1, 'ids': []}]}]


## Submitting problem to the Azure Quantum solver

The SlcTerm functionality is only available on the [Population Annealing](https://docs.microsoft.com/azure/quantum/optimization-population-annealing#parameterized-population-annealing) and [Substochastic Monte Carlo](https://docs.microsoft.com/azure/quantum/optimization-substochastic-monte-carlo#parameterized-substochastic-monte-carlo) solvers. For this sample we will use the population annealing solver with its default parameters (since the problem is trivial).



In [16]:
solver = PopulationAnnealing(
    workspace,
    sweeps=10,
    seed=10
)

print('Submitting QAP...')
start = time.time()
result = solver.optimize(problem)
timeElapsed = time.time() - start
print('Result in {:.1f} seconds: '.format(timeElapsed), result["solutions"][0])

Submitting QAP...
....Result in 3.7 seconds:  {'configuration': {'0': 1, '1': 0, '2': 0, '3': 0, '4': 1, '5': 0, '6': 0, '7': 0, '8': 1}, 'cost': 270.0}


We can see that the cost (270) matches the optimal solution that we found at the beginning of this exercise. 

The solver returns a configuration mapped to the binary variables, but to interpret this, we need to map them back to the original variable definitions. 

In [17]:
solution = result["solutions"][0]
for v in variable_map:
    var_id = variable_map[v]
    if solution["configuration"][str(var_id)] == 1:
        print(f"Facility {v[0]} assigned to Location {v[1]}")
        

Facility 0 assigned to Location 0
Facility 1 assigned to Location 1
Facility 2 assigned to Location 2


## Further Resources
A public list of larger quadratic assignment problems can be found at [Cor\@l](https://coral.ise.lehigh.edu/data-sets/qaplib/) . As an exercise, you can try converting them into Azure Quantum QIO form using the same method described above.
